In [1]:
import sys
sys.path.insert(1, '..')
from bokeh.io import show, output_notebook
from bokeh.layouts import column,row
output_notebook(hide_banner=True)
from cocoa import covid19 as co
from cocoa import display as cd
from cocoa import world as cw
import pandas as pd
w=cw.WorldInfo()

# SPF database choosen 

In [2]:
import importlib
importlib.reload(co)
importlib.reload(cd)
spf = co.DataBase('spf')
coco = cd.CocoDisplay()

SPF aka Sante Publique France database selected ...
... tree differents db from SPF will be parsed ...
Few information concernant the selected database :  spf
Available keys words for:  ['hosp', 'rea', 'rad', 'dc', 'incid_hosp', 'incid_rea', 'incid_dc', 'incid_rad', 'P', 'T', 'tx_incid', 'R', 'taux_occupation_sae', 'tx_pos']
Example of location :  58, 58, 94, 34, 53  ...
Last date data  11/3/20


In [3]:
location = w.getAllFranceDepartment()
def returnpd(val,date):
    a = spf.get_stats(which=val,location=location)
    mask = (a['date'] <=  date) & (a['date'] >=  "2020-09-01")
    a = a.loc[mask]
    a = a.groupby('date').agg({val: "sum"})
    print('sum', val , sum(a[val]))
date1='2020-10-10'
date2='2020-10-20'
print(date1)
returnpd('P',date1) 
returnpd('T',date1)
returnpd('incid_hosp',date1)
returnpd('incid_rea',date1)
returnpd('incid_dc',date1)
print(date2)
returnpd('P',date2) 
returnpd('T',date2)
returnpd('incid_hosp',date2)
returnpd('incid_rea',date2)
returnpd('incid_dc',date2)


2020-10-10
sum P 449937.0
sum T 5755533.0
sum incid_hosp 20857.0
sum incid_rea 3785.0
sum incid_dc 1749.0
2020-10-20
sum P 713383.0
sum T 7487841.0
sum incid_hosp 32257.0
sum incid_rea 5748.0
sum incid_dc 2776.0


# Plot 'indice reanimation' (incid_rea) for île de France

In [4]:
sp=spf.get_stats(which='taux_occupation_sae',location=w.getIleDeFrance()[0])
show(coco.cocoa_basic_plot(sp,'taux_occupation_sae'))

# Plot "indice décés" (incid_dc) for île de France
Nombre quotidien de personnes nouvellement décédées

In [5]:
sp=spf.get_stats(which='incid_dc',location=w.getIleDeFrance()[0])
sp['smoothed'] =  sp['incid_dc'].rolling(7).mean()
show(coco.cocoa_basic_plot(sp,'smoothed'))

# Plot 'indice décés' cumulé (indic_dc) for île de France
Nombre total de de personnes nouvellement décédées

In [6]:
show(coco.cocoa_basic_plot(sp,'cumul'))

# Plot "dc" cumulé (dc) for île de France
Total amout of deaths at the hospital

In [7]:
sp=spf.get_stats(which='dc',location=w.getIleDeFrance()[0])
show(coco.cocoa_basic_plot(sp,'dc'))

In [8]:
sp_dc=spf.get_stats(which='dc',location=['13','75'])
sp_cumul=spf.get_stats(which='incid_dc',location=['13','75'])
sp_dc['cumul_incid_dc'] = sp_cumul['cumul']
show(coco.cocoa_basic_plot(sp_dc,'dc'))

In [9]:
ta=spf.get_stats(which='taux_occupation_sae',location=['75','13'])
show(coco.cocoa_basic_plot(ta,'taux_occupation_sae'))

In [10]:
def retrieve_R(loc):
    sp=spf.get_stats(which='R',location=loc)
    sp = sp.groupby('date').agg({'R': "mean"})
    sp = sp.reset_index()
    df = sp[['date', 'R']]
    df = df.dropna()
    return df.set_index('date')

location = w.getAllFranceDepartment()
a = spf.get_stats(which='incid_hosp',location=location)
a = a.groupby('date').agg({'incid_hosp': "sum"})

smoothed = spf.smooth_cases(a.stack())
posteriors = spf.get_posteriors(smoothed)
posteriors = posteriors.idxmax()
a['Reff'] =  posteriors.unstack()['incid_hosp']

df = retrieve_R(location)
result=pd.concat([a,df], axis=1, sort=False)

fig=coco.cocoa_basic_plot(result.dropna(),['Reff','R'])
show(fig)

# Function to resume data

In [11]:
def resumeplot(loc):
    if type(loc) is list:
        location = loc
        name = 'France'
    if type(loc) is str:
        location = loc
        name = loc
    if type(loc) is tuple: 
        location = loc[0]
        name = loc[1]
    
    sp=spf.get_stats(which='rea',location=location)
    sp = sp.groupby('date').agg({'rea': "sum"})
    sp['rea'] = sp['rea'].rolling(7).mean()
    fig1=coco.cocoa_basic_plot(sp,['rea'],title='Rea ' + name +' moyenne glissante sur 7 jours')

    dc=spf.get_stats(which='incid_dc',location=location)
    dc = dc.groupby('date').agg({'incid_dc': "sum"})
    dc['incid_dc'] = dc['incid_dc'].rolling(7).mean()
    fig2=coco.cocoa_basic_plot(dc,['incid_dc'],title='Deces (hopital) ' + name +' moyenne glissante sur 7 jours')
    
    po=spf.get_stats(which='P',location=location)
    po = po.groupby('date').agg({'P': "sum"})
    po['P'] = po['P'].rolling(7).mean()

    te=spf.get_stats(which='T',location=location)
    te = te.groupby('date').agg({'T': "sum"})
    te['T'] = te['T'].rolling(7).mean()
    po['positivite'] = po['P']/te['T'] 
    fig3=coco.cocoa_basic_plot(po,['positivite'],title='Taux Positivité ' + name +' moyenne glissante sur 7 jours')
    
    ta=spf.get_stats(which='taux_occupation_sae',location=location)
    ta = ta.groupby('date').agg({'taux_occupation_sae': "mean"})
    ta['taux_occupation_sae'] = ta['taux_occupation_sae'].rolling(7).mean()
    fig4=coco.cocoa_basic_plot(ta,['taux_occupation_sae'],title='Taux d’occupation des lits en réanimation par des patients COVID par rapport à la capacité initiale en réanimation'
                                        + name +' moyenne glissante sur 7 jours')
 
    show(column(row(fig1,fig2),row(fig3,fig4)))

# Admissions en réa moyenne glissante sur 7 jours, toute la France
(identique à article de catherine hill )

In [12]:
resumeplot(w.getAllFranceDepartment())

# Admissions en réa île deFrance, moyenne glissante sur 7 jours

In [62]:
resumeplot(w.getIleDeFrance())

In [63]:
resumeplot(w.getProvenceAlpesCotedAzur())

# Statistiques hospitalières
can be compared with https://www.cascoronavirus.fr/stats/france/region/ile-de-france

# Île-de-France  

In [64]:
from datetime import datetime, timedelta
loc = w.getIleDeFrance()[0]
rea = spf.get_stats(which='rea',location=loc)

indice_rea = spf.get_stats(which='incid_rea',location=loc)
mask2 = (indice_rea['date'] == indice_rea['date'].max()) # '2020-10-24'
mask1 = (indice_rea['date'] == indice_rea['date'].max()-timedelta(days=1)) #'2020-10-23'
reacum1 = indice_rea.loc[mask1]
reacum2 = indice_rea.loc[mask2]

print("Au jour de ",  indice_rea['date'].max())
print("Réanimations au total depuis le début de l'épidémie :",sum(reacum2['cumul']))
print("Réanimations en cours:",sum(rea.loc[mask2]['rea']))
print("En 24h :", sum(rea.loc[mask2]['rea']) - sum(rea.loc[mask1]['rea']))

Au jour de  2020-11-03 00:00:00
Réanimations au total depuis le début de l'épidémie : 10161.0
Réanimations en cours: 987.0
En 24h : 26.0


In [16]:
rea = spf.get_stats(which='rea',location=loc)
show(coco.cocoa_basic_plot(rea,['rea']))

In [17]:
from pandas import DataFrame
rea = spf.get_stats(which='rea',location=loc)
maxdate=rea['date'].max()
mindate=rea['date'].min()
oneweeklist={}
current_date = maxdate
i=0
while current_date > mindate:
    current_date = maxdate-i*timedelta(days=1)
    mask2p = (rea['date'] == current_date)
    mask1p = (rea['date'] == current_date-timedelta(days=1))
    reacum2 = rea.loc[mask2p]
    reacum1 = rea.loc[mask1p]          
    oneweeklist[rea['date'].max()-i*timedelta(days=1)]=sum(reacum2['cumul']) - sum(reacum1['cumul'])
    i+=1
       
df = pd.DataFrame(oneweeklist.items(), columns=['date', 'Diff'])
df = df.set_index('date')

print(df)
show(coco.cocoa_basic_plot(df,'Diff',"Différence des entrées en réanimation d'une semaine sur l'autre en île de France"))


             Diff
date             
2020-11-03    0.0
2020-11-02  961.0
2020-11-01  920.0
2020-10-31  901.0
2020-10-30  912.0
...           ...
2020-03-22  640.0
2020-03-21  545.0
2020-03-20  484.0
2020-03-19  360.0
2020-03-18  294.0

[231 rows x 1 columns]


# Hospitalisations au total depuis le début de l'épidémie 

In [65]:
loc = w.getIleDeFrance()[0]
hosp = spf.get_stats(which='dc',location=loc)
print(hosp)
mask2 = (hosp['date'] == hosp['date'].max()) # '2020-10-24'
mask1 = (hosp['date'] == hosp['date'].max()-timedelta(days=1)) #'2020-10-23'

hospcum2 = hosp.loc[mask2]
hospcum1 = hosp.loc[mask1]
print(hospcum2)
print("DC au total depuis le début de l'épidémie :",hospcum2['dc'].sum())
print("en 24h :", hospcum2['dc'].sum()-hospcum1['dc'].sum())

    location       date     dc     cumul  diff
0         75 2020-03-18   14.0      14.0   0.0
1         75 2020-03-19   22.0      36.0   8.0
2         75 2020-03-20   30.0      66.0   8.0
3         75 2020-03-21   36.0     102.0   6.0
4         75 2020-03-22   40.0     142.0   4.0
..       ...        ...    ...       ...   ...
226       95 2020-10-30  865.0  144441.0   2.0
227       95 2020-10-31  873.0  145314.0   8.0
228       95 2020-11-01  879.0  146193.0   6.0
229       95 2020-11-02  885.0  147078.0   6.0
230       95 2020-11-03  893.0  147971.0   8.0

[1848 rows x 5 columns]
    location       date      dc     cumul  diff
230       75 2020-11-03  2131.0  366565.0  15.0
230       77 2020-11-03   838.0  138504.0   8.0
230       78 2020-11-03   670.0  109067.0  13.0
230       91 2020-11-03   741.0  112209.0  11.0
230       92 2020-11-03  1280.0  222111.0   9.0
230       93 2020-11-03  1215.0  207183.0   4.0
230       94 2020-11-03  1426.0  244617.0  10.0
230       95 2020-11-03   8

In [66]:
hosp=hosp.groupby('date').agg({'dc': "sum"})
show(coco.cocoa_basic_plot(hosp,['dc']))

# DC France depuis le début de l'épidémie 

In [72]:
loc = w.getAllFranceDepartment()

dc = spf.get_stats(which='dc',location=loc)
mask2 = (dc['date'] == dc['date'].max()) # '2020-10-24'
mask1 = (dc['date'] == dc['date'].max()-timedelta(days=1)) #'2020-10-23'
dccum2 = dc.loc[mask2]
dccum1 = dc.loc[mask1]

print("DC au total depuis le début de l'épidémie :",dccum2['dc'].sum())
print("en 24h :", dccum2['dc'].sum()-dccum1['dc'].sum())

DC au total depuis le début de l'épidémie : 26186.0
en 24h : 426.0


In [75]:
hosp = spf.get_stats(which='hosp',location=loc)
hospcum2 = hosp.loc[mask2]
hospcum1 = hosp.loc[mask1]
print("Hospitalisations au total depuis le début de l'épidémie :",hospcum2['hosp'].sum())
print("en 24h :", hospcum2['hosp'].sum()-hospcum1['hosp'].sum())

Hospitalisations au total depuis le début de l'épidémie : 26242.0
en 24h : 1122.0


In [74]:
rea = spf.get_stats(which='rea',location=loc)
reacum2 = rea.loc[mask2]
reacum1 = rea.loc[mask1]
print("Rea au total depuis le début de l'épidémie :",reacum2['rea'].sum())
print("en 24h :", reacum2['rea'].sum()-reacum1['rea'].sum())

Rea au total depuis le début de l'épidémie : 3869.0
en 24h : 148.0


In [71]:
gue = spf.get_stats(which='rad',location=loc)

guecum2 = gue.loc[mask2]
guecum1 = gue.loc[mask1]

print("Gueris après hospitalisations au total depuis le début de l'épidémie :",guecum2['rad'].sum())
print("en 24h :", guecum2['rad'].sum()-guecum1['rad'].sum())

Gueris après hospitalisations au total depuis le début de l'épidémie : 120683.0
en 24h : 1610.0


# Reff France (méthode de Cori) 
https://covidtracker.fr/covidtracker-france/

Méthode de calcul basée sur la méthode de Cori :
R = A / B

A = nb. de personnes admises aux urgences pour Covid19 (sur 7 jours)
B = nb. de personnes admises aux urgences pour Covid19 il y a 7 jours (sur 7 jours)

In [25]:
location = w.getAllFranceDepartment()
a = spf.get_stats(which='incid_hosp',location=location,output='pandas')
a = a.groupby('date').agg({'incid_hosp': "sum"})
a['incid_hosp']=a['incid_hosp'].rolling(7).mean()
a['incid_hosp7']= a['incid_hosp'].shift(7)

a['ratio'] = a['incid_hosp']/a['incid_hosp7']
a['location'] = 'France'
fig=coco.cocoa_basic_plot(a,['ratio'])
show(fig)